In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import datetime

In [2]:
# Read the data file
df=pd.read_csv('LoanStats3b_securev1.csv',header=1)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,49,129,130,131,134,135,136,139) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Choose some meaningful and available features for predictions
df=df[['loan_amnt','term','int_rate','installment','sub_grade','emp_title','emp_length','home_ownership','annual_inc','verification_status',
        'issue_d','purpose','zip_code','addr_state','dti','delinq_2yrs','earliest_cr_line','fico_range_low','fico_range_high','inq_last_6mths',
        'open_acc','revol_bal','revol_util','total_acc','acc_open_past_24mths','mort_acc','percent_bc_gt_75','loan_status']]


In [4]:
# Drop rows that are completely empty
df.dropna(how='all', inplace=True)

In [5]:
# Change the format of int_rate to float
df['int_rate']=df['int_rate'].apply(lambda x: float(x[:-1]))

In [6]:
# Change the format of revol_util to float
def process_revol_util(x):
    if type(x) == str:
        x=float(x[:-1])
    return x
df['revol_util']=df['revol_util'].apply(lambda x: process_revol_util(x))


In [7]:
# Fill in missing values with the mean
df['revol_util'].fillna(value=df['revol_util'].mean(), inplace=True)
df['acc_open_past_24mths'].fillna(value=df['acc_open_past_24mths'].mean(), inplace=True)
df['mort_acc'].fillna(value=df['mort_acc'].mean(), inplace=True)
df['percent_bc_gt_75'].fillna(value=df['percent_bc_gt_75'].mean(), inplace=True)

In [8]:
# Change emp_length and sub_grade from strings to integers
mapping_dict={"emp_length": {"10+ years": 10, "9 years": 9, "8 years": 8, "7 years": 7, "6 years": 6, "5 years": 5, "4 years": 4, 
                               "3 years": 3, "2 years": 2, "1 year": 1, "< 1 year": 0}, 
                "sub_grade":{"G5": 0, "G4": 1, "G3": 2, "G2": 3, "G1": 4, "F5": 5, "F4": 6, "F3": 7, "F2": 8, "F1": 9, "E5": 10,
                             "E4": 11, "E3": 12, "E2": 13, "E1": 14, "D5": 15, "D4": 16, "D3": 17, "D2": 18, "D1": 19, "C5": 20, 
                             "C4": 21, "C3": 22, "C2": 23, "C1": 24, "B5": 25, "B4": 26, "B3": 27, "B2": 28, "B1": 29, "A5": 30, 
                             "A4": 31, "A3": 32, "A2": 33, "A1": 34}}
df=df.replace(mapping_dict)


In [9]:
# Fill in missing values of emp_length based on emp_title
for i in df.index:
    if np.isnan(df.loc[i,'emp_length']):
        title=df.loc[i,'emp_title']
        if type(title)==str:
            average=df[df['emp_title']==title]['emp_length'].mean()
            df.loc[i,'emp_length']=average

In [10]:
# Drop the rows with NaN in either emp_title or emp_length
df.dropna(subset=['emp_title','emp_length'], inplace=True)

In [11]:
# Change issue_d to datetime format
df['issue_d']=df['issue_d'].apply(lambda x: datetime.datetime.strptime(x, '%b-%Y'))

In [12]:
# Change earliest_cr_line to datetime format
df['earliest_cr_line']=df['earliest_cr_line'].apply(lambda x: datetime.datetime.strptime(x, '%b-%Y'))

In [ ]:
# Calculate the differences in months from earliest_cr_line to issue_d
def diff_month(d1,d2):
    return (d1.year-d2.year)*12+(d1.month-d2.month)

for i in df.index:
    df.loc[i,'earliest_cr_line']=diff_month(df.loc[i,'issue_d'],df.loc[i,'earliest_cr_line'])

In [18]:
# Creat dummy variables for the rest categorical variables
dummy_df=pd.get_dummies(df[['term','home_ownership','verification_status','purpose','addr_state']])
df=pd.concat([df, dummy_df], axis=1)
df=df.drop(['term','home_ownership','verification_status','purpose','addr_state'], axis=1)

In [ ]:
# Change the target variable to numbers: 1 indicates defaulted, 0 indicates paid off
def flag(x):
    if x=='Fully Paid':
        return 0
    elif x=='Charged Off':
        return 1

df['loan_status']=df['loan_status'].apply(flag)

# Drop loans with other unknown status
df.dropna(subset=['loan_status'], inplace=True)

In [ ]:
# Reset the index to integers starting from 0
df.reset_index(drop=True, inplace=True)

In [19]:
df

,loan_amnt,int_rate,installment,sub_grade,emp_title,emp_length,annual_inc,issue_d,zip_code,dti,...,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY
0,12000.0,7.62,373.94,32,Systems Engineer,3.0,96500.0,2013-12-01,782xx,12.61,...,0,0,1,0,0,0,0,0,0,0
1,27050.0,10.99,885.46,28,Team Leadern Customer Ops & Systems,10.0,55000.0,2013-12-01,481xx,22.87,...,0,0,0,0,0,0,0,0,0,0
2,12000.0,11.99,398.52,27,LTC,10.0,130000.0,2013-12-01,809xx,13.03,...,0,0,0,0,0,0,0,0,0,0
3,28000.0,7.62,872.52,32,Area Sales Manager,5.0,325000.0,2013-12-01,945xx,18.55,...,0,0,0,0,0,0,0,0,0,0
4,27600.0,19.97,730.78,15,Street Operations Supervisor,6.0,73000.0,2013-12-01,802xx,23.13,...,0,0,0,0,0,0,0,0,0,0
5,12000.0,10.99,392.81,28,Project Manager,4.0,60000.0,2013-12-01,281xx,4.62,...,0,0,0,0,0,0,0,0,0,0
6,11100.0,14.98,384.68,22,Teacher,10.0,90000.0,2013-12-01,103xx,3.73,...,0,0,0,0,0,0,0,0,0,0
7,12000.0,13.53,407.40,25,On road manager,10.0,40000.0,2013-12-01,871xx,16.94,...,0,0,0,0,0,0,0,0,0,0
8,9750.0,13.98,333.14,24,Medical Assistant,1.0,26000.0,2013-12-01,927xx,25.12,...,0,0,0,0,0,0,0,0,0,0
9,4800.0,10.99,157.13,28,Surgical Technician,2.0,39600.0,2013-12-01,782xx,2.49,...,0,0,1,0,0,0,0,0,0,0


In [37]:
df.dtypes

loan_amnt                  float64
int_rate                   float64
installment                float64
sub_grade                    int64
emp_title                   object
emp_length                 float64
annual_inc                 float64
issue_d                     object
zip_code                    object
dti                        float64
delinq_2yrs                float64
earliest_cr_line             int64
fico_range_low             float64
fico_range_high            float64
inq_last_6mths             float64
open_acc                   float64
revol_bal                  float64
revol_util                 float64
total_acc                  float64
acc_open_past_24mths       float64
mort_acc                   float64
percent_bc_gt_75           float64
loan_status                float64
term_ 36 months              int64
term_ 60 months              int64
home_ownership_MORTGAGE      int64
home_ownership_NONE          int64
home_ownership_OTHER         int64
home_ownership_OWN  

In [36]:
df.isnull().sum()

loan_amnt                  0
int_rate                   0
installment                0
sub_grade                  0
emp_title                  0
emp_length                 0
annual_inc                 0
issue_d                    0
zip_code                   0
dti                        0
delinq_2yrs                0
earliest_cr_line           0
fico_range_low             0
fico_range_high            0
inq_last_6mths             0
open_acc                   0
revol_bal                  0
revol_util                 0
total_acc                  0
acc_open_past_24mths       0
mort_acc                   0
percent_bc_gt_75           0
loan_status                0
term_ 36 months            0
term_ 60 months            0
home_ownership_MORTGAGE    0
home_ownership_NONE        0
home_ownership_OTHER       0
home_ownership_OWN         0
home_ownership_RENT        0
                          ..
addr_state_MA              0
addr_state_MD              0
addr_state_MI              0
addr_state_MN 

In [22]:
df.describe()

,loan_amnt,int_rate,installment,sub_grade,emp_length,annual_inc,dti,delinq_2yrs,earliest_cr_line,fico_range_low,...,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY
count,176415.000000,176415.000000,176415.000000,176415.000000,176415.000000,1.764150e+05,176415.000000,176415.000000,176415.000000,176415.000000,...,176415.000000,176415.000000,176415.000000,176415.000000,176415.000000,176415.000000,176415.000000,176415.000000,176415.000000,176415.000000
mean,14439.926735,14.292174,445.677114,23.242083,6.071540,7.273147e+04,17.071844,0.241244,183.714010,696.700422,...,0.002171,0.010764,0.078179,0.007947,0.031188,0.001593,0.023978,0.012284,0.004710,0.002477
std,8087.465306,4.437909,241.198616,6.412320,3.570892,5.164721e+04,7.566998,0.706035,81.788926,29.800178,...,0.046544,0.103192,0.268454,0.088792,0.173826,0.039879,0.152980,0.110149,0.068471,0.049709
min,1000.000000,6.000000,4.930000,0.000000,0.000000,5.000000e+03,0.000000,0.000000,36.000000,660.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8000.000000,11.140000,273.850000,20.000000,3.000000,4.500000e+04,11.380000,0.000000,129.000000,675.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,12375.000000,14.090000,399.260000,24.000000,6.000000,6.300000e+04,16.800000,0.000000,168.000000,690.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,20000.000000,17.270000,579.000000,28.000000,10.000000,8.800000e+04,22.560000,0.000000,225.000000,710.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,35000.000000,26.060000,1407.010000,34.000000,10.000000,7.141778e+06,34.990000,29.000000,750.000000,845.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [23]:
df.to_csv('2012-2013_cleaned.csv')

In [3]:
df=pd.read_csv('2012-2013_cleaned.csv', index_col=0)

In [8]:
corrdf=df.iloc[:,:22].corr()

In [10]:
corrdf[corrdf>0.4]

,loan_amnt,int_rate,installment,sub_grade,emp_length,annual_inc,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,open_acc,revol_bal,revol_util,total_acc,acc_open_past_24mths,mort_acc,percent_bc_gt_75
loan_amnt,1.00000,NaN,0.95415,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
int_rate,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
installment,0.95415,NaN,1.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub_grade,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.519093,0.519089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
emp_length,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
annual_inc,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
dti,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
delinq_2yrs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
earliest_cr_line,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fico_range_low,NaN,NaN,NaN,0.519093,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
HSIC

In [4]:
import matplotlib

In [5]:
matplotlib.__version__

'3.0.0'